***Mediana del precio de oferta***


In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [13]:
import re
# Lista de nombres de DataFrames
nombres_df = [
    'df_alfa_abril_2024_queretaro',
    'df_alfa_agosto_2024_queretaro',
    'df_alfa_marzo_2024_queretaro',
    'df_alfa_mayo_2024_queretaro',
    'df_junio_2023_queretaro'
]

# Diccionario para almacenar las medianas de precios por mes
medianas_precio = {}

# Iterar sobre los nombres de DataFrames y calcular la mediana de 'precio'
for nombre_df in nombres_df:
    # Obtener el DataFrame por su nombre
    df = globals()[nombre_df]
    
    # Usar regex para extraer mes y año del nombre del DataFrame
    match = re.search(r'alfa_(\w+)_(\d{4})', nombre_df)  # Buscar el patrón de mes y año
    if match:
        mes = match.group(1)
        año = match.group(2)
        
        # Formar el nombre para el diccionario
        mes_año = f"{mes}_{año}"
        
        # Calcular la mediana de la columna 'precio' y almacenarla
        medianas_precio[mes_año] = df['precio'].median()

# Ver las medianas calculadas
print(medianas_precio)


{'abril_2024': 4500000.0, 'agosto_2024': 4100000.0, 'marzo_2024': 4500000.0, 'mayo_2024': 4500000.0}


In [14]:
# 1. Mediana de junio_2024 (promedio de mayo y agosto)
mediana_junio_2024 = (medianas_precio['mayo_2024'] + medianas_precio['agosto_2024']) / 2
# 2. Mediana de julio_2024 (promedio de junio y agosto)
mediana_julio_2024 = (mediana_junio_2024 + medianas_precio['agosto_2024']) / 2
# Añadir los valores calculados al diccionario
medianas_precio['junio_2024'] = mediana_junio_2024
medianas_precio['julio_2024'] = mediana_julio_2024
print(medianas_precio)

{'abril_2024': 4500000.0, 'agosto_2024': 4100000.0, 'marzo_2024': 4500000.0, 'mayo_2024': 4500000.0, 'junio_2024': 4300000.0, 'julio_2024': 4200000.0}


In [15]:
# Convertir el diccionario en listas ordenadas por meses
meses = ['Marzo 2024', 'Abril 2024', 'Mayo 2024', 'Junio 2024', 'Julio 2024', 'Agosto 2024']
precios = [
    medianas_precio['marzo_2024'],
    medianas_precio['abril_2024'],
    medianas_precio['mayo_2024'],
    medianas_precio['junio_2024'],
    medianas_precio['julio_2024'],
    medianas_precio['agosto_2024']
]

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=meses, y=precios, mode='lines+markers', line=dict(color='blue')))
# Actualizar el diseño del gráfico
fig.update_layout(
    title='Histórico de precios (2024)',
    xaxis_title='Meses',
    yaxis_title='Precio',
    legend_title="Segmento",
    xaxis=dict(tickangle=45),
    yaxis=dict(gridcolor='#dddcda'),
    legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=50, b=10)
)
fig.show()

In [20]:
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_precio_oferta_mediana', carpeta='assets/graficas')